In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# ==============================================================================
# EJERCICIO 2: ESTUDIO DETALLADO Y APLICACIÓN DE SVM (Support Vector Machine)
# ==============================================================================

"""
--- DOCUMENTACIÓN MEJORADA DEL ALGORITMO SVM ---

1. ¿QUÉ ES SVM?
   Es un algoritmo de aprendizaje supervisado potente para clasificación y regresión.
   Su objetivo fundamental es encontrar el HIPERPLANO óptimo que separa las clases
   en el espacio de características.

2. CONCEPTOS CLAVE:

   A. Hiperplano:
      Es la frontera de decisión. En 2D es una línea, en 3D un plano.
      Todo lo que cae a un lado se clasifica como Clase A, y al otro como Clase B.

   B. Vectores de Soporte (Support Vectors):
      Son los puntos de datos (instancias) más difíciles de clasificar porque
      están más cerca de la línea divisoria. Son los puntos "críticos".
      El algoritmo SVM se llama así porque se apoya exclusivamente en estos puntos
      para definir el hiperplano; los puntos lejanos no importan.

   C. Margen (Margin):
      Es la distancia entre el hiperplano y los vectores de soporte más cercanos.
      SVM busca MAXIMIZAR este margen. Un margen más amplio significa que el modelo
      es más robusto y generaliza mejor.

   D. Kernel Trick (Truco del Kernel):
      ¿Qué pasa si los datos no se pueden separar con una línea recta (no son linealmente separables)?
      SVM proyecta los datos a una dimensión superior (ej: de 2D a 3D) donde sí
      se puedan separar linealmente.
      - Linear: Para datos simples.
      - RBF (Radial Basis Function): El más popular para fronteras curvas complejas.
      - Polynomial: Crea fronteras curvas de grado específico.

   E. Hiperparámetros Críticos (C y Gamma):
      - C (Regularización):
        - C alto: Intenta clasificar todo correctamente (riesgo de Overfitting/Sobreajuste). Margen estrecho.
        - C bajo: Permite algunos errores para buscar un margen más ancho (Underfitting/Suave).
      - Gamma (Solo para kernels RBF/Poly):
        - Gamma alto: Considera solo puntos muy cercanos al hiperplano (fronteras muy curvas y ajustadas).
        - Gamma bajo: Considera puntos lejanos (fronteras más suaves y rectas).
"""

# ==============================================================================
# APLICACIÓN PRÁCTICA: CLASIFICACIÓN DE VINOS
# ==============================================================================

# 1. Carga de Datos
# Usamos el dataset de Vinos (Wine Dataset), un problema clásico multiclase.
# 13 características químicas (alcohol, málico, cenizas...) para clasificar 3 tipos de vinos.
wine = datasets.load_wine()
X = wine.data
y = wine.target
target_names = wine.target_names

print(f"Dataset cargado: {X.shape[0]} muestras, {X.shape[1]} características.")
print(f"Clases de vino: {target_names}")

# 2. Preprocesamiento
# SVM es MUY sensible a la escala de los datos. Si una característica tiene valores de 1000
# y otra de 0.1, la de 1000 dominará el cálculo de distancias. ES OBLIGATORIO ESCALAR.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Configuración y Entrenamiento del Modelo SVM
# Probamos con un Kernel RBF que es versátil para datos reales complejos.
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

print("\nEntrenando SVM con kernel RBF...")
svm_model.fit(X_train_scaled, y_train)

# 4. Evaluación
y_pred = svm_model.predict(X_test_scaled)

print("\n--- REPORTE DE CLASIFICACIÓN ---")
print(classification_report(y_test, y_pred, target_names=target_names))

acc = accuracy_score(y_test, y_pred)
print(f"Precisión Global (Accuracy): {acc*100:.2f}%")

# ==============================================================================
# VISUALIZACIÓN (Reduciendo a 2 dimensiones para poder ver el Hiperplano)
# ==============================================================================
# Para visualizar los márgenes, entrenaremos un nuevo SVM usando solo las 2 primeras features.
X_vis = X_train_scaled[:, :2]  # Solo Alcohol y Ácido Málico
y_vis = y_train

svm_vis = SVC(kernel='linear', C=1.0) # Usamos linear aquí para ver líneas claras
svm_vis.fit(X_vis, y_vis)

# Crear malla para plotear
x_min, x_max = X_vis[:, 0].min() - 1, X_vis[:, 0].max() + 1
y_min, y_max = X_vis[:, 1].min() - 1, X_vis[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))

Z = svm_vis.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.coolwarm)
plt.scatter(X_vis[:, 0], X_vis[:, 1], c=y_vis, edgecolors='k', cmap=plt.cm.coolwarm)
plt.title('SVM: Fronteras de Decisión (Solo 2 características para visualización)')
plt.xlabel('Alcohol (Estandarizado)')
plt.ylabel('Ácido Málico (Estandarizado)')
plt.show()

print("\nCONCLUSIÓN SVM:")
print("El algoritmo logra separar eficazmente las clases maximizando el margen.")
print("La estandarización (StandardScaler) fue el paso crítico para obtener buenos resultados.")